In [1]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [11]:
import pandas as pd
import re
import os
from datetime import datetime, timedelta

# --- 1. 配置区域 ---
FILE_PATH = r"D:\5507课件\作业\ppmt超话帖子.csv"
SAVE_PATH = r"D:\5507课件\作业\ppmt超话帖子_2025清洗版.csv"

# 【核心时间设置】
# 设定“今天”为用户指定的 2025-11-23
CURRENT_DATE_SIMULATION = datetime(2025, 11, 23)
TARGET_YEAR = 2025

# 筛选区间
FILTER_START = datetime(2025, 6, 1)
FILTER_END = datetime(2025, 11, 23, 23, 59, 59) # 包含当天的最后一秒

def clean_weibo_content(text):
    """
    清洗内容：
    1. 去除  (乱码占位符)
    2. 去除首尾空白
    3. 替换多余的换行符
    """
    if not isinstance(text, str):
        return ""
    
    # 去除具体的乱码字符
    text = text.replace('', '').replace('?', '') # 有些乱码显示为?
    
    # 去除不可见字符 (如 \u200b)
    text = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)
    
    # 规范化空白
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def standardize_time_force_2025(time_str):
    """
    时间解析函数（强制适配2025年）
    """
    if not isinstance(time_str, str) or not time_str.strip():
        return pd.NaT
    
    time_str = time_str.strip()
    
    try:
        # 情况1: "2025-06-01" (完整日期)
        if re.match(r'^\d{4}-\d{1,2}-\d{1,2}', time_str):
            return pd.to_datetime(time_str)
            
        # 情况2: "06-01 12:00" (缺年份 -> 强制补全为 2025)
        elif re.match(r'^\d{1,2}-\d{1,2}\s+\d{1,2}:\d{1,2}', time_str):
            full_time_str = f"{TARGET_YEAR}-{time_str}"
            return pd.to_datetime(full_time_str)
            
        # 情况3: "今天 12:00" (基准日为 2025-11-23)
        elif "今天" in time_str:
            time_part = re.search(r'\d{1,2}:\d{1,2}', time_str).group()
            full_time_str = f"{CURRENT_DATE_SIMULATION.strftime('%Y-%m-%d')} {time_part}"
            return pd.to_datetime(full_time_str)
            
        # 情况4: "昨天 12:00"
        elif "昨天" in time_str:
            yesterday = CURRENT_DATE_SIMULATION - timedelta(days=1)
            time_part = re.search(r'\d{1,2}:\d{1,2}', time_str).group()
            full_time_str = f"{yesterday.strftime('%Y-%m-%d')} {time_part}"
            return pd.to_datetime(full_time_str)

        # 情况5: "10分钟前", "刚刚" (视为 2025-11-23 当前时刻)
        elif "分钟前" in time_str or "刚刚" in time_str or "秒前" in time_str:
            return CURRENT_DATE_SIMULATION
            
        else:
            return pd.NaT
            
    except:
        return pd.NaT

def main():
    print(f"正在读取原始文件: {FILE_PATH}")
    if not os.path.exists(FILE_PATH):
        print("错误：文件不存在！")
        return

    # 1. 读取数据
    try:
        df = pd.read_csv(FILE_PATH, encoding='utf-8-sig')
    except:
        try:
            df = pd.read_csv(FILE_PATH, encoding='gbk')
        except Exception as e:
            print(f"文件读取失败: {e}")
            return

    print(f"原始数据总数: {len(df)} 条")

    # 2. 清洗微博内容 (去除乱码)
    print("正在清洗内容乱码...")
    df['微博内容'] = df['微博内容'].apply(clean_weibo_content)

    # 3. 标准化时间 (核心步骤)
    print(f"正在解析时间 (年份强制锁定为 {TARGET_YEAR})...")
    df['parsed_time'] = df['发布时间'].apply(standardize_time_force_2025)

    # 4. 删除无效时间 (侧边栏、广告等)
    df = df.dropna(subset=['parsed_time'])

    # 5. 按照指定区间过滤数据
    # 2025-06-01 <= 时间 <= 2025-11-23
    print(f"正在筛选区间: {FILTER_START.strftime('%Y-%m-%d')} 至 {FILTER_END.strftime('%Y-%m-%d')}")
    
    mask = (df['parsed_time'] >= FILTER_START) & (df['parsed_time'] <= FILTER_END)
    df_filtered = df.loc[mask].copy()
    
    dropped_count = len(df) - len(df_filtered)
    if len(df_filtered) == 0:
        print("="*50)
        print("【警告】筛选结果为 0 条！")
        print("可能原因：原始CSV里的数据实际上是2024年的，或者日期格式无法识别。")
        print("请检查原始CSV中的‘发布时间’列长什么样。")
        print("="*50)
        return

    # 6. 排序 (倒序：最新 -> 最旧)
    print("正在按时间倒序排列...")
    df_filtered = df_filtered.sort_values(by='parsed_time', ascending=False)

    # 7. 整理并保存
    # 将解析好的标准时间写回发布时间列
    df_filtered['发布时间'] = df_filtered['parsed_time']
    df_filtered = df_filtered.drop(columns=['parsed_time']) # 删除辅助列

    print(f"正在保存清洗后的文件: {SAVE_PATH}")
    df_filtered.to_csv(SAVE_PATH, index=False, encoding='utf-8-sig')

    print("\n" + "="*30)
    print("清洗任务完成！")
    print(f"保留有效数据: {len(df_filtered)} 条")
    print(f"剔除无效/非区间数据: {dropped_count} 条")
    print("="*30)

if __name__ == "__main__":
    main()

正在读取原始文件: D:\5507课件\作业\ppmt超话帖子.csv
原始数据总数: 4188 条
正在清洗内容乱码...
正在解析时间 (年份强制锁定为 2025)...
正在筛选区间: 2025-06-01 至 2025-11-23
正在按时间倒序排列...
正在保存清洗后的文件: D:\5507课件\作业\ppmt超话帖子_2025清洗版.csv

清洗任务完成！
保留有效数据: 4121 条
剔除无效/非区间数据: 67 条
